# This notebook is to Implement my solution for the home assignment
* Written by: Yotam Dery
* Submission date: 05/16/2024

## Imports

In [1]:
import numpy as np
import pandas as pd
import warnings
from statsmodels.tsa.seasonal import seasonal_decompose
from utils import aggregate_duplicates, perform_adf_test_all_features
from plot_utils import plot_feature_over_time, plot_combined_trends, plot_corr, box_plot, plot_seasonal_decomposition, plot_lag_plots, plot_acf_plots, plot_mean_std_plots

warnings.filterwarnings('ignore')

## EDA

### Reading the data file

In [2]:
# Reading and showing glimps of the data
df = pd.read_csv('lightricks_interview_dataset.csv', parse_dates=['month'])
df.tail(15)

,month,spend,revenue,subs
45,2024-01-01,78245.433186,228361.488025,42807.797696
46,2024-01-01,79194.849350,228361.488025,42807.797696
47,2024-02-01,72942.356713,NaN,NaN
48,2024-02-01,68365.845848,NaN,NaN
49,2024-03-01,84123.965592,NaN,NaN
50,2024-03-01,76110.299450,NaN,NaN
51,2024-04-01,78180.634086,NaN,NaN
52,2024-05-01,86814.089306,NaN,NaN
53,2024-06-01,87459.192145,NaN,NaN
54,2024-07-01,86602.082479,NaN,NaN


In [3]:
# As I noticed that there are rows which are duplicated by their month value - we handle it in this function
aggregated_df = aggregate_duplicates(df)

In [4]:
# Check for missing values
print(aggregated_df.isnull().sum())
print("\nrevenue and the number of subscribers is not known ahead of time. Hence, These null values are O.K.")

month       0
spend       0
revenue    11
subs       11
dtype: int64

revenue and the number of subscribers is not known ahead of time. Hence, These null values are O.K.


In [5]:
aggregated_df.describe()

,spend,revenue,subs
count,45.000000,34.000000,34.000000
mean,76008.017483,189175.002446,43183.683014
std,15068.365802,58885.541838,11815.383900
min,53974.044782,102986.770996,21775.164505
25%,68682.142098,141739.270826,33865.645563
50%,73281.546183,177698.244333,43103.881024
75%,80117.132521,237638.505176,52467.379499
max,138713.913887,304535.260688,65055.140810


In [6]:
aggregated_df.dtypes

month      datetime64[ns]
spend             float64
revenue           float64
subs              float64
dtype: object

### Plot time series plots

In [7]:
for feature in aggregated_df.select_dtypes(include='number'):
    plot_feature_over_time(aggregated_df, feature)

In [8]:
# Create traces
plot_combined_trends(aggregated_df)

### Correlation Analysis

In [9]:
correlation_matrix = np.round(aggregated_df.corr(),2)
plot_corr(correlation_matrix)
print("As observed from the time series plots and confirmed by the correlation analysis,\nthe features revenue and subscriptions are highly correlated.")

As observed from the time series plots and confirmed by the correlation analysis,
the features revenue and subscriptions are highly correlated.


### Box Plot

In [10]:
for feature in aggregated_df.select_dtypes(include='number'):
    box_plot(aggregated_df, feature)
print("As we can see, there are no outliers according to the box plot.\nThe boxes are fairly even")

As we can see, there are no outliers according to the box plot.
The boxes are fairly even


### Seasonal Decomposition

* Seasonal decomposition is a technique used in time series analysis to decompose a time series into several components, each representing an underlying pattern.<br> It helps in understanding and analyzing the structure of the time series data by separating it into more interpretable elements.<br> The main components in seasonal decomposition are:<br>
1. Trend Component:<br> This captures the long-term progression of the time series data. It represents the overall direction (upward or downward) in the data over a long period.<br><br>

2. Seasonal Component:<br> This captures the repeating short-term cycle in the data. For example, monthly sales data may have a seasonal component that reflects higher sales in certain months each year. <br><br>

3. Residual (or Irregular) Component:<br> This captures the random noise or irregular fluctuations in the data that cannot be explained by the trend or seasonal components. It's essentially the remaining part of the time series after removing the trend and seasonal effects.<br><br>

When applying this method, we'll use an Additive Model, Assuming that the time series is composed of the sum of its components: <br>
`Y(t)=T(t)+S(t)+R(t)`


In [11]:
# Set 'month' as index
aggregated_df.dropna(inplace=True)
aggregated_df.set_index('month', inplace=True)

In [12]:
for feature in aggregated_df.select_dtypes(include='number'):
    # Perform seasonal decomposition
    result = seasonal_decompose(aggregated_df[feature], model='additive', period=12)
    plot_seasonal_decomposition(result, aggregated_df, feature)

We can clearly draw some interesting insights from these plots: <br><br>
* Seasonality:<br>
    1. For Spend and Revenue, we can see that there are peaks every year around the summer of each year (May-October), and lows in the other months. <br> 
    2. For subscribers, the seasonality is much more narrow, peaking arround August each year, with lows in the winter (lowest at Feb.) <br><br>
* Residual: <br>
    The residual plot shows the residual component, which represents the random noise or irregular fluctuations in the data.<br>
    For all metrics, the residuals fluctuate around zero, indicating the presence of random variations not explained by the trend or seasonal components. <br> Some periods show higher residuals (positive or negative), suggesting occasional unexpected increases or decreases in spend that are not part of the regular pattern.
    

### Lag plots
* The lag plot is used to identify the presence of autocorrelation in a time series dataset. It plots each data point 
`y(t)` against the next data point `y(t+1)`.<br> High autocorrelation can indicate non-stationarity. 

In [13]:
for feature in df.select_dtypes(include='number'):
    plot_lag_plots(aggregated_df, feature, lag=1)

* Insights: <br><br>
1. Positive Autocorrelation: Spend shows a positive autocorrelation, with similar values tending to follow each other.<br> This metric is higher in Revenue and Subscribers features<br><br>
2. Variability: There is variability in spend, but a clustering around certain values suggests some level of stability or common spend ranges.<br>

In [14]:
for feature in df.select_dtypes(include='number'):
    plot_lag_plots(aggregated_df, feature, lag=12)

* Insights: <br><br>
1. Positive Autocorrelation: Taking a lag period of 12, Revenue shows a weak autocorrelation, while the other features show no autocorrelation.<br><br>
2. Variability: There is high variability in all features. This Implies on a low level of predictability.

### ACF Plot

In [15]:
for feature in df.select_dtypes(include='number'):
    plot_acf_plots(aggregated_df, feature)

* <b>General observations</b><br>
1. Stationarity: All three series ('spend', 'revenue', and 'subs') show signs of non-stationarity due to significant autocorrelations at initial lags. This non-stationarity could be due to trends or persistence in the data. <br>
2. To achieve stationarity, differencing the time series data is recommended. This involves subtracting the previous observation from the current observation to remove trends and stabilize the mean.<br>
3. The significant autocorrelations suggest that ARIMA models could be suitable for forecasting these time series. The AR (AutoRegressive) component of the model will help capture the persistence seen in the data.

### Rolling Mean and Standard Deviation Plot

In [16]:
plot_mean_std_plots(aggregated_df, window_size=12)

* Insights and Strategic Implications: <br><br>
1. Sustained Growth: The steady increase in revenue and subscriptions indicates strong overall business performance, driven by effective strategies in acquiring and retaining customers.<br>
2. Predictability and Stability: Low and stable variability in both revenue and subscriptions, coupled with controlled spending, indicates a predictable and stable business environment, which is beneficial for long-term planning and forecasting.<br>
3. Balanced Approach: The data reflects a balanced approach where initial investments (higher spend) were made to drive growth (increased subscriptions and revenue), followed by a phase of optimized spending to maintain sustainable growth.<br>
4. The presence of a trend in the rolling mean and variations in the rolling standard deviation suggest that the features data is non-stationary (futher investigation should be conducted on that manner).

#### Performing ADF test for stationarity check

In [17]:
# Perform ADF test on the each features' data
perform_adf_test_all_features(aggregated_df, aggregated_df.select_dtypes(include='number'))

ADF test results:

Test results for feature: Spend
Used lag: 1
ADF Statistic: -4.862993591691974
p-value: 4.1144488726076716e-05
The time series is stationary.
##########################################


Test results for feature: Revenue
Used lag: 5
ADF Statistic: -0.7384520974658513
p-value: 0.8365366421484501
The time series is non-stationary.
##########################################


Test results for feature: Subs
Used lag: 0
ADF Statistic: -1.9985458298241614
p-value: 0.2872110658934871
The time series is non-stationary.
##########################################



* Not all data is stationary. Let's us diff on the data and test for stationarity again:

In [18]:
# Calculating the diff
for feature in aggregated_df.select_dtypes(include='number'):
    aggregated_df[feature+'_diff'] = aggregated_df[feature].diff()

aggregated_df.dropna(inplace=True)
aggregated_df.reset_index(inplace=True)

In [19]:
# Creating a list containing only the diff features
diff_features_list = []
for feature in aggregated_df.select_dtypes(include='number'):
    if '_diff' in feature:
        diff_features_list.append(feature)

# Plotting the diff features
for feature in diff_features_list:
    plot_feature_over_time(aggregated_df, feature)

In [20]:
# Perform ADF test on the each features' data
perform_adf_test_all_features(aggregated_df, diff_features_list)

ADF test results:

Test results for feature: Spend_diff
Used lag: 2
ADF Statistic: -4.1994660251091105
p-value: 0.0006602728975336425
The time series is stationary.
##########################################


Test results for feature: Revenue_diff
Used lag: 3
ADF Statistic: -4.04671125944463
p-value: 0.0011847300641754607
The time series is stationary.
##########################################


Test results for feature: Subs_diff
Used lag: 0
ADF Statistic: -6.33902514597515
p-value: 2.7842565680743006e-08
The time series is stationary.
##########################################



* Looking at the recent plots and the test above, it seems like our data after differencing is stationary!

In [21]:
aggregated_df

,month,spend,revenue,subs,spend_diff,revenue_diff,subs_diff
0,2021-05-01,53974.044782,136158.778966,26374.078293,-3926.118724,29503.359694,4598.913788
1,2021-06-01,60858.720356,102986.770996,27977.380806,6884.675574,-33172.007970,1603.302513
2,2021-07-01,65244.047917,121084.866299,28152.685280,4385.327561,18098.095303,175.304474
3,2021-08-01,121780.554145,142439.980755,30049.015135,56536.506228,21355.114455,1896.329855
4,2021-09-01,138713.913887,165528.474944,34778.395575,16933.359743,23088.494189,4729.380440
5,2021-10-01,93205.197946,174315.310926,35137.150866,-45508.715942,8786.835982,358.755291
6,2021-11-01,57711.989462,134474.850878,33775.384796,-35493.208484,-39840.460049,-1361.766070
7,2021-12-01,79046.396583,163836.430411,35741.190631,21334.407121,29361.579533,1965.805834
8,2022-01-01,82125.520527,133168.574076,32739.408334,3079.123945,-30667.856335,-3001.782297
9,2022-02-01,73984.395431,113051.580013,31738.391192,-8141.125096,-20116.994063,-1001.017142


In [22]:
df

,month,spend,revenue,subs
0,2021-04-01,57900.163506,106655.419272,21775.164505
1,2021-05-01,53974.044782,136158.778966,26374.078293
2,2021-06-01,60858.720356,102986.770996,27977.380806
3,2021-07-01,65244.047917,121084.866299,28152.685280
4,2021-08-01,121780.554145,142439.980755,30049.015135
5,2021-09-01,138713.913887,165528.474944,34778.395575
6,2021-10-01,93205.197946,174315.310926,35137.150866
7,2021-11-01,57711.989462,134474.850878,33775.384796
8,2021-12-01,79046.396583,163836.430411,35741.190631
9,2022-01-01,82125.520527,133168.574076,32739.408334
